# Capstone: Evaluating Housing Program Effectiveness in Tooele County
### **An analysis of public housing program effectiveness** in meeting the needs of low- and moderate-income residents in Tooele County, Utah.
### Research Question:
> "Will the projected housing prices in Tooele County, Utah, in 2025 increase at a greater rate than the projected income growth of local residents?"
### Hypothesis:
> "Projected housing prices of available housing stock in 2025 will increase at a greater rate than the projected income of residents in Tooele County, Utah."

In [ ]:
# import pandas as pd       # For data manipulation
# import numpy as np        # For numerical operations

# import requests           # For accessing REST APIs
# import json               # For working with JSON responses

# from urllib.parse import urlencode  # For formatting API query strings
# from datetime import datetime        # For date parsing and manipulation

# import matplotlib.pyplot as plt      # Core plotting library
# import seaborn as sns                # Statistical visualization
# import plotly.express as px          # Interactive plots (optional but great for storytelling)

# import os                            # For directory and file path operations

# import scipy.stats as stats          # For significance testing
# import statsmodels.api as sm         # For regression or modeling (optional)

# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, r2_score

# %matplotlib inline

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import kagglehub
import glob

## 1. Introduction
- **Research Question:** How effective are housing assistance programs in Tooele County at supporting low- and moderate-income families?
- **Focus Areas:** Section 8, Family Self-Sufficiency, Permanent Supportive Housing, Homeownership Programs

## 📚 1.2. Background and Literature Review

I. Housing Affordability: 2024 Utah Priorities Project  
   https://www.utahfoundation.org/reports/housing-affordability-2024-priorities-brief-1/

II. American Community Survey (ACS) – U.S. Census Bureau  
   https://www.census.gov/programs-surveys/acs/

III. National Association of Realtors Housing Forecast  
   https://www.nar.realtor/research-and-statistics/housing-statistics


## 2. Data Acquisition
- Sources:
> - **Income Data**: U.S. Census Bureau (ACS API)
> - **Housing Prices**: Zillow ZHVI via Kaggle or scraped CSV
> - **Employment Data**: BLS API
> - **Local Economics**: Utah DWS

### API Calls and Static Data Downloads

In [ ]:
# Income Data: Census API call for median income

CENSUS_API_KEY = "de53f7491b5574a451233a6d04721c4176263ab4"
census_url = f"https://api.census.gov/data/2022/acs/acs5?get=NAME,B19013_001E&for=county:045&in=state:49&key={CENSUS_API_KEY}"

response = requests.get(census_url)
income_data = response.json()
income_df = pd.DataFrame(income_data[1:], columns=income_data[0])
income_df.head()

In [ ]:
# Housing Prices – Zillow ZHVI (Zillow Home Value Index)

path = kagglehub.dataset_download("robikscube/zillow-home-value-index")
print("Path to dataset files:", path)

# Load all home price index data
zhvi_path = os.path.join(path, "State_ZHVI_AllHomes.csv")
zhvi_df = pd.read_csv(zhvi_path)

# Filter for Tooele County, Utah
tooele_df = zhvi_df[(zhvi_df['RegionName'] == 'Tooele County') & (zhvi_df['State'] == 'UT')]
tooele_df.head()

In [ ]:
# Employment Data: Bureau of Labor Statistics (BLS) API

headers = {'Content-type': 'application/json'}
data = json.dumps({
  "seriesid": ["LAUCN490450000000003"],  # Replace with relevant series ID
  "startyear": "2015",
  "endyear": "2024",
  "registrationkey": "6563620fb1c04af6ab1809ebe75012e2"
})

response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
bls_data = response.json()

In [ ]:
# Local Economics – Utah Department of Workforce Services (DWS)

# Set the path to your data folder
data_dir = 'data'
pattern = os.path.join(data_dir, 'industry-and-wages-data-*.csv')

# Collect all matching file paths
file_paths = glob.glob(pattern)

# Read and combine all CSVs, adding a 'Year' column extracted from the filename
df_list = []
for file_path in file_paths:
    year = int(os.path.basename(file_path).split('-')[-1].split('.')[0])  # Extract year from filename
    df = pd.read_csv(file_path)
    df['Year'] = year
    df_list.append(df)

# Merge into a single DataFrame
local_df = pd.concat(df_list, ignore_index=True)

# Optional: Sort by Year
local_df.sort_values(by='Year', inplace=True)

# View result
print(local_df.head())

## 3. Data Cleaning & Preparation
- Drop nulls, standardize formats, join datasets
- Feature engineering: affordability ratios, utilization rates

## 4. Exploratory Data Analysis (EDA)
- Trends in rent, income, voucher use
- Maps and charts

## 5. Program-Level Analysis
### 5.1 Section 8 & Tenant Protection
### 5.2 Family Self-Sufficiency
### 5.3 Permanent Supportive Housing
### 5.4 Homeownership Programs

## 6. Evaluation Metrics
- Rent-to-income ratio
- Voucher utilization
- Program coverage

## 7. Conclusions and Recommendations
- Summary of findings
- Policy insights

## 8. Code Appendix
- Functions and utilities